In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.models import load_model # type: ignore

input_shape = (64, 64, 3)

base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid')
])

# model = load_model('checkpoint/modelo_BN.keras')


dataset_dir = "/Users/jorgecamero/MoireDataset/BN_aug"

train_data = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    labels="inferred",
    label_mode="int",
    class_names=["Reales", "Ataque"],
    batch_size=1,
    image_size=(64, 64),
    validation_split=0.2,
    subset="training",
    seed=123
)

val_data = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    labels="inferred",
    label_mode="int",
    class_names=["Reales", "Ataque"],
    batch_size=1,
    image_size=(64, 64),
    validation_split=0.2,
    subset="validation",
    seed=123
)

# Configuración de AdamW con la tasa de aprendizaje especificada
optimizer = tf.keras.optimizers.AdamW(learning_rate=1e-4)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_data, epochs=50, validation_data=val_data)

In [ ]:
model.save("checkpoint/modelo_BN.keras")

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

HEIGHT = 100
WIDTH = 175

path = '/Users/jorgecamero/MoireDataset/BN_aug'
folders = [os.path.join(path, folder) for folder in os.listdir(path) if os.path.isdir(os.path.join(path, folder))]
random_folder = random.choice(folders)

img_list = [os.path.join(random_folder, file) for file in os.listdir(random_folder) if file.lower().endswith(('.jpg', '.png', '.jpeg'))]
random_img = random.choice(img_list)

img = image.load_img(random_img, target_size=(64, 64))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
prediction = model.predict(img, verbose=False)[0][0]
print(prediction)
prediction_label = "Color" if prediction < 0.5 else "BN"

print(prediction_label)
plt.imshow(load_img(random_img, width=64, height=64))
plt.title(f'{os.path.basename(random_folder)} {os.path.basename(random_img)}')
plt.axis('off')
plt.show()


In [ ]:
import os
import shutil
from tqdm import tqdm
from keras.models import load_model # type: ignore
from tensorflow.keras.preprocessing import image
from IPython.display import clear_output
import matplotlib.pyplot as plt

#model = load_model('checkpoint/modelo_BN.keras')
path = '/Users/jorgecamero/MoireDataset/BN_aug/Reales'

img_list = [os.path.join(path, file) for file in os.listdir(path) if file.lower().endswith(('.jpg', '.png', '.jpeg'))]
pause_time = 0.5

for img_path in tqdm(img_list, desc="Procesando imágenes"):
    img = image.load_img(img_path, target_size=(32, 32))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img, verbose=False)[0]
    prediction_bin = 0 if prediction < 0.5 else 1

    if prediction_bin:
        filename = os.path.basename(img_path)
        plt.imshow(load_img(img_path))
        plt.title(filename)
        plt.axis('off')
        print(prediction)
        plt.draw()
        plt.pause(pause_time)
        clear_output(wait=True)


In [ ]:
from PIL import Image
import os

# Ruta de la carpeta de origen y la de destino
input_folder = '/Users/jorgecamero/MoireDataset/BN_aug/Reales'
output_folder = '/Users/jorgecamero/MoireDataset/BN_aug/Ataque'

# Crear la carpeta de destino si no existe
os.makedirs(output_folder, exist_ok=True)

# Iterar sobre cada imagen en la carpeta
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.jpg', '.png', '.jpeg')):
        img_path = os.path.join(input_folder, filename)
        
        # Abrir la imagen
        img = Image.open(img_path)
        
        # Convertir a blanco y negro (grayscale)
        img_bw = img.convert("L")
        
        # Convertir la imagen en escala de grises a RGB
        img_bw_rgb = img_bw.convert("RGB")
        
        # Guardar la imagen en la carpeta de destino
        img_bw_rgb.save(os.path.join(output_folder, filename))

print("Conversión completada.")